In [1]:
import mitosheet

/opt/conda/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (5.0.0)/charset_normalizer (2.0.9) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [3]:
mitosheet.sheet(pd.read_csv(ocd_file, low_memory=False))

MitoWidget(analysis_data_json='{"analysisName": "id-nmopxskrrv", "analysisToReplay": null, "code": [], "stepSu…